# Playing with OpenID Connect flows against a Keykloack setup

This is a Python3 notebook that illustrates different OpenID Connect flows, using a local Keycloak instance as OpenID provider and some basic libraries to handle the HTTP interactions.

You can skip the set up part and go straight to the flows:

- [Client Credentials Flow](#Client-Credentials-Flow)
- [Resource Owner Password Flow](#Resource-Owner-Password-Flow)
- [Authorization Code Flow](#Authorization-Code-Flow)



In [1]:
import base64
import html
import json
import logging
import re
import urllib.parse
import uuid

import requests

In [2]:
logging.basicConfig(level=logging.INFO)

## Setup of Keycloak instance

We need a test/development Keycloak instance.
For example, spin up a local Keycloak instance with Docker as follows:

    docker run --rm -it -p 9090:8080  \
        -e KEYCLOAK_USER=admin -e KEYCLOAK_PASSWORD=admin  \
        jboss/keycloak:7.0.0

In [3]:
keycloak_base_url = "http://localhost:9090/auth"
admin_username = "admin"
admin_password = "admin"

### Keycloak Admin API

To be able to create clients and users through the Keycloak admin API, we first have to obtain an admin access token through OpenID, which we have to use a bearer token for other admin API requests. Let's wrap this stuff in a class.

In [4]:
class KeycloakAdmin:
    def __init__(self, base_url: str, username: str, password: str):
        r = requests.post(
            base_url + '/realms/master/protocol/openid-connect/token', 
            data={
                "username": username,
                "password": password,
                "grant_type": "password",
                "client_id": "admin-cli"
            })
        r.raise_for_status()
        
        self.session = requests.Session()
        self.session.headers["Authorization"]= "Bearer " + r.json()["access_token"]
        self.admin_base_url = base_url + '/admin/realms/master'
        self.log = logging.getLogger("keycloak-admin")
    
    def create_client(self, options: dict = None, prefix: str = "myclient-") -> str:
        client_id = prefix + uuid.uuid4().hex[:8]
        data = {"id": client_id}
        data.update(options)
        self.log.info("Creating client with settings {s!r}".format(s=data))
        r = self.session.post(self.admin_base_url + '/clients', json=data)
        r.raise_for_status()
        return client_id

    def get_client_secret(self, client_id) -> str:
        r = self.session.get(self.admin_base_url + '/clients/{c}/client-secret'.format(c=client_id))
        r.raise_for_status()
        self.log.info("Client secret response: {r!r}".format(r=r.text))
        client_secret = r.json()["value"]
        return client_secret
    
    def create_user(self, prefix: str = "John-", password: str = "j0hn"):
        username = prefix + uuid.uuid4().hex[:8]

        r = self.session.post(
            self.admin_base_url + '/users', 
            json={
                "username": username,
                "credentials": [
                    {"type": "password", "value": password, "temporary": False},
                ],
                "enabled": True,
            }
        )
        r.raise_for_status()
        return username, password


# And while we're at it,
def jwt_decode(token: str):
    """Poor man's JWT decoding"""

    def _decode(data: str) -> dict:
        decoded = base64.b64decode(data + '=' * (4 - len(data) % 4)).decode('ascii')
        return json.loads(decoded)

    header, payload, signature = token.split('.')
    return _decode(header), _decode(payload)

In [5]:
keycloak_admin = KeycloakAdmin(keycloak_base_url, admin_username, admin_password)

## General Set Up

To better see what is going on the HTTP level when doing OpenID Connect request, we'll add a `requests` hook that prints a bit of request and response info.

In [6]:
from IPython.display import HTML, display

def _show_request_info(r, *args, **kwargs):
    req = r.request
    default_headers = requests.utils.default_headers()
    headers = {k:v for k,v in req.headers.items() if k not in default_headers}
    display(HTML('''<div style="padding: 1ex; border: 1px solid #ddf; background-color: #eef;">
        Did request: <code>{m} {u}</code> with <ul>
        <li>body <code>{b!r}</code></li>
        <li>headers <code>{h!r}</code></li>
        <li>&rArr; response {r}</li>
        </ul></div>'''.format(
        m=req.method, u=req.url, 
        b=req.body, h=headers,
        r=r.status_code
    )))

session = requests.Session()
session.hooks['response'].append(_show_request_info)

And while we're at it, define some additional small utilities.

In [7]:
def jwt_decode(token: str):
    """Poor man's JWT decoding"""

    def _decode(data: str) -> dict:
        decoded = base64.b64decode(data + '=' * (4 - len(data) % 4)).decode('ascii')
        return json.loads(decoded)

    header, payload, signature = token.split('.')
    return _decode(header), _decode(payload)

## OpenID provider info

Get OpenID provider info from the configuration document.

In [8]:
provider_info = session.get(
    keycloak_base_url + '/realms/master/.well-known/openid-configuration'
).json()
provider_info.keys()

dict_keys(['issuer', 'authorization_endpoint', 'token_endpoint', 'token_introspection_endpoint', 'userinfo_endpoint', 'end_session_endpoint', 'jwks_uri', 'check_session_iframe', 'grant_types_supported', 'response_types_supported', 'subject_types_supported', 'id_token_signing_alg_values_supported', 'id_token_encryption_alg_values_supported', 'id_token_encryption_enc_values_supported', 'userinfo_signing_alg_values_supported', 'request_object_signing_alg_values_supported', 'response_modes_supported', 'registration_endpoint', 'token_endpoint_auth_methods_supported', 'token_endpoint_auth_signing_alg_values_supported', 'claims_supported', 'claim_types_supported', 'claims_parameter_supported', 'scopes_supported', 'request_parameter_supported', 'request_uri_parameter_supported', 'code_challenge_methods_supported', 'tls_client_certificate_bound_access_tokens', 'introspection_endpoint'])

Get the token endpoint URL.

In [9]:
token_endpoint = provider_info["token_endpoint"]
token_endpoint

'http://localhost:9090/auth/realms/master/protocol/openid-connect/token'

# Client Credentials Flow

Create a client in Keycloak with settings that allow enable Client Credentials Grant. We'll also need the client's secret.

In [10]:
cc_client = keycloak_admin.create_client(options={
    "serviceAccountsEnabled": True,
})
cc_client_secret = keycloak_admin.get_client_secret(cc_client)

cc_client, cc_client_secret

INFO:keycloak-admin:Creating client with settings {'id': 'myclient-d56e28d8', 'serviceAccountsEnabled': True}
INFO:keycloak-admin:Client secret response: '{"type":"secret","value":"c00ed59d-87ea-4b2c-b1e7-036287b51483"}'


('myclient-d56e28d8', 'c00ed59d-87ea-4b2c-b1e7-036287b51483')

Do `client_credentials` token request.

In [11]:
r = session.post(
    token_endpoint,
    data={
        "grant_type": "client_credentials",
        "client_id": cc_client,
        "client_secret": cc_client_secret,
    }
)
r.raise_for_status()
r.json()

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJXVEdTMXJyN1pTY2RWU05DV1d5M0tiVUJXNFVaUDc2ZFZ1V1l4RTdYSnYwIn0.eyJqdGkiOiI5YzMzZGM1OS1kMTNiLTQ0ODgtODdjYi0wNDVmYjFlNzBmYzQiLCJleHAiOjE1NzE3MzU1NDUsIm5iZiI6MCwiaWF0IjoxNTcxNzM1NDg1LCJpc3MiOiJodHRwOi8vbG9jYWxob3N0OjkwOTAvYXV0aC9yZWFsbXMvbWFzdGVyIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImRkYjJlM2JhLWEyZjItNDYzMC04YWUzLTU5ZGU2Y2ViYmFmNCIsInR5cCI6IkJlYXJlciIsImF6cCI6Im15Y2xpZW50LWQ1NmUyOGQ4IiwiYXV0aF90aW1lIjowLCJzZXNzaW9uX3N0YXRlIjoiM2JjNmRjYzUtMGRiMi00ZWEzLWE3YmItNjI4Njk3Y2RmYmEyIiwiYWNyIjoiMSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJlbWFpbCBwcm9maWxlIiwiY2xpZW50SG9zdCI6IjE3Mi4xNy4wLjEiLCJlbWFpbF92ZXJpZmllZCI6ZmFsc2UsImNsaWVudElkIjoibXljbGllbnQtZDU2ZTI4ZDgiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJzZXJ2aWNlLWFjY291bnQtbXljbGllbnQtZDU2ZTI4ZDgiLCJjbGllbnRBZGRyZX

## JWT inspection

Extract access token and inspect it (assuming it is a JWT token).

In [12]:
access_token = r.json()["access_token"]
jwt_decode(access_token)

({'alg': 'RS256',
  'typ': 'JWT',
  'kid': 'WTGS1rr7ZScdVSNCWWy3KbUBW4UZP76dVuWYxE7XJv0'},
 {'jti': '9c33dc59-d13b-4488-87cb-045fb1e70fc4',
  'exp': 1571735545,
  'nbf': 0,
  'iat': 1571735485,
  'iss': 'http://localhost:9090/auth/realms/master',
  'aud': 'account',
  'sub': 'ddb2e3ba-a2f2-4630-8ae3-59de6cebbaf4',
  'typ': 'Bearer',
  'azp': 'myclient-d56e28d8',
  'auth_time': 0,
  'session_state': '3bc6dcc5-0db2-4ea3-a7bb-628697cdfba2',
  'acr': '1',
  'realm_access': {'roles': ['offline_access', 'uma_authorization']},
  'resource_access': {'account': {'roles': ['manage-account',
     'manage-account-links',
     'view-profile']}},
  'scope': 'email profile',
  'clientHost': '172.17.0.1',
  'email_verified': False,
  'clientId': 'myclient-d56e28d8',
  'preferred_username': 'service-account-myclient-d56e28d8',
  'clientAddress': '172.17.0.1',
  'email': 'service-account-myclient-d56e28d8@placeholder.org'})

## Query `userinfo`

Check the access token against the `userinfo` endpoint

In [13]:
r = session.get(
    provider_info["userinfo_endpoint"], 
    headers={"Authorization": "Bearer %s" % access_token}
)
r.raise_for_status()
r.json()

{'sub': 'ddb2e3ba-a2f2-4630-8ae3-59de6cebbaf4',
 'email_verified': False,
 'preferred_username': 'service-account-myclient-d56e28d8',
 'email': 'service-account-myclient-d56e28d8@placeholder.org'}

# Resource Owner Password Flow

Create a client that allows resource owner password flow.

In [14]:
pwd_client = keycloak_admin.create_client({
    "publicClient": True,
    "directAccessGrantsEnabled": True,
})
pwd_client

INFO:keycloak-admin:Creating client with settings {'id': 'myclient-c5d5aa6e', 'publicClient': True, 'directAccessGrantsEnabled': True}


'myclient-c5d5aa6e'

In [15]:
user, password = keycloak_admin.create_user()

In [16]:
r = session.post(
    token_endpoint,
    data={
        "grant_type": "password",
        "username": user,
        "password": password,
        "client_id": pwd_client,
    }
)
r.raise_for_status()
r.json()

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJXVEdTMXJyN1pTY2RWU05DV1d5M0tiVUJXNFVaUDc2ZFZ1V1l4RTdYSnYwIn0.eyJqdGkiOiJlNDUxYTE4Yy1lMmIzLTQ5MzctOTk0NS01Y2E5NGUwMGI5ZTAiLCJleHAiOjE1NzE3MzU1NDUsIm5iZiI6MCwiaWF0IjoxNTcxNzM1NDg1LCJpc3MiOiJodHRwOi8vbG9jYWxob3N0OjkwOTAvYXV0aC9yZWFsbXMvbWFzdGVyIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6ImRhMjRjZDBmLWQxNTktNDI3OC1hN2FkLTI2NmU4YjQ2YTNhNCIsInR5cCI6IkJlYXJlciIsImF6cCI6Im15Y2xpZW50LWM1ZDVhYTZlIiwiYXV0aF90aW1lIjowLCJzZXNzaW9uX3N0YXRlIjoiYWNkZGEyNGYtY2NiNC00OWFiLWEzNDgtZTA4OTU5ZDRlYzgzIiwiYWNyIjoiMSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJlbWFpbCBwcm9maWxlIiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJqb2huLTBkN2U1NmU3In0.YaThzzLn5HD_ISSymDaYoCDhaP26yRDVo6JqgLb1g4nok-spb2ts8varQQmezUM0iC5hKPVhlXm5u48DrylQKOMHMdBO7DRgTedn_CXRd-IXGC-v4QTYHl

## JWT inspection

Extract access token and inspect it (assuming it is a JWT token).

In [17]:
access_token = r.json()["access_token"]
jwt_decode(access_token)

({'alg': 'RS256',
  'typ': 'JWT',
  'kid': 'WTGS1rr7ZScdVSNCWWy3KbUBW4UZP76dVuWYxE7XJv0'},
 {'jti': 'e451a18c-e2b3-4937-9945-5ca94e00b9e0',
  'exp': 1571735545,
  'nbf': 0,
  'iat': 1571735485,
  'iss': 'http://localhost:9090/auth/realms/master',
  'aud': 'account',
  'sub': 'da24cd0f-d159-4278-a7ad-266e8b46a3a4',
  'typ': 'Bearer',
  'azp': 'myclient-c5d5aa6e',
  'auth_time': 0,
  'session_state': 'acdda24f-ccb4-49ab-a348-e08959d4ec83',
  'acr': '1',
  'realm_access': {'roles': ['offline_access', 'uma_authorization']},
  'resource_access': {'account': {'roles': ['manage-account',
     'manage-account-links',
     'view-profile']}},
  'scope': 'email profile',
  'email_verified': False,
  'preferred_username': 'john-0d7e56e7'})

## Query `userinfo`

Check the access token against the `userinfo` endpoint.

In [18]:
r = session.get(
    provider_info["userinfo_endpoint"], 
    headers={"Authorization": "Bearer %s" % access_token}
)
r.raise_for_status()
r.json()

{'sub': 'da24cd0f-d159-4278-a7ad-266e8b46a3a4',
 'email_verified': False,
 'preferred_username': 'john-0d7e56e7'}

# Authorization Code Flow

Create a client that allows the Authorization Code flow

In [19]:
redirect_uri = "https://example.com/redir"
ac_client = keycloak_admin.create_client({
    "publicClient": False,
    "redirectUris": [redirect_uri]
})
ac_client_secret = keycloak_admin.get_client_secret(ac_client)
ac_client

INFO:keycloak-admin:Creating client with settings {'id': 'myclient-f312da4e', 'publicClient': False, 'redirectUris': ['https://example.com/redir']}
INFO:keycloak-admin:Client secret response: '{"type":"secret","value":"056c2ccc-0660-49c4-bfa4-b4f8b948622b"}'


'myclient-f312da4e'

And create a user

In [20]:
user, password = keycloak_admin.create_user()

Start Authorization Code flow: we are forwarded to the OpenID provider (log in page).

In [21]:
r = session.get(
    url=provider_info["authorization_endpoint"],
    params={
        "response_type": "code",
        "client_id": ac_client,
        "scope": "openid",
        "redirect_uri": redirect_uri,
        "state": "foobar",
    },
    headers={},
    allow_redirects=False
)
r.raise_for_status()

Extract the form action so we can submit the form (the requests session will take care of the cookies).

In [22]:
form_action = html.unescape(re.search('<form\s+.*?\s+action="(.*?)"', r.text, re.DOTALL).group(1))
form_action

'http://localhost:9090/auth/realms/master/login-actions/authenticate?session_code=5er8NTT4yVUMTX3b6rGiR1U5o08oZanswDhc0ZYz0EQ&execution=fdf3c29c-a1f3-4d24-82f8-00fb7a8b8115&client_id=myclient-f312da4e&tab_id=g1pa56BrQes'

Log in, capture redirect and extract authorization code.

In [23]:
r = session.post(
    form_action, 
    data={"username": user, "password": password},
    allow_redirects=False
)
assert r.status_code == 302
redirect = r.headers['Location']
print(redirect)
session.cookies.clear()

redirect_params = urllib.parse.parse_qs(urllib.parse.urlparse(redirect).query)
auth_code = redirect_params["code"]
auth_code

https://example.com/redir?state=foobar&session_state=28e891db-dedb-4c69-9502-83a985c8a384&code=93d6a6da-ba59-4dd5-90ed-22002469c3eb.28e891db-dedb-4c69-9502-83a985c8a384.myclient-f312da4e


['93d6a6da-ba59-4dd5-90ed-22002469c3eb.28e891db-dedb-4c69-9502-83a985c8a384.myclient-f312da4e']

Exchange authorization code for an access token.

In [24]:
r = session.post(
    url=token_endpoint,
    data={
        "grant_type": "authorization_code",
        "client_id": ac_client,
        "client_secret": ac_client_secret,
        "redirect_uri": redirect_uri,
        "code": auth_code,
    },
    allow_redirects=False
)
r.raise_for_status()
r.json()

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJXVEdTMXJyN1pTY2RWU05DV1d5M0tiVUJXNFVaUDc2ZFZ1V1l4RTdYSnYwIn0.eyJqdGkiOiJiYjI3NzU3Ni0zZDg5LTQzMDAtODJlMy05NmY0NmQ0NjczYTQiLCJleHAiOjE1NzE3MzU1NDYsIm5iZiI6MCwiaWF0IjoxNTcxNzM1NDg2LCJpc3MiOiJodHRwOi8vbG9jYWxob3N0OjkwOTAvYXV0aC9yZWFsbXMvbWFzdGVyIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6IjQ4OTI1N2JkLTdiMDQtNDNlYy1hNDJjLTVjMDlhZTU0NDQ1YiIsInR5cCI6IkJlYXJlciIsImF6cCI6Im15Y2xpZW50LWYzMTJkYTRlIiwiYXV0aF90aW1lIjoxNTcxNzM1NDg2LCJzZXNzaW9uX3N0YXRlIjoiMjhlODkxZGItZGVkYi00YzY5LTk1MDItODNhOTg1YzhhMzg0IiwiYWNyIjoiMSIsImFsbG93ZWQtb3JpZ2lucyI6WyJodHRwczovL2V4YW1wbGUuY29tIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicHJlZmVycmVkX3VzZXJuYW1lIjoiam9obi1lMTNiNTVlMyJ9.XQzRweuZKFdkTAugNvJr7r_9J6tHzT3RcK_nqDxt2

## JWT inspection

Extract access token and inspect it (assuming it is a JWT token).

In [25]:
access_token = r.json()["access_token"]

jwt_decode(access_token)

({'alg': 'RS256',
  'typ': 'JWT',
  'kid': 'WTGS1rr7ZScdVSNCWWy3KbUBW4UZP76dVuWYxE7XJv0'},
 {'jti': 'bb277576-3d89-4300-82e3-96f46d4673a4',
  'exp': 1571735546,
  'nbf': 0,
  'iat': 1571735486,
  'iss': 'http://localhost:9090/auth/realms/master',
  'aud': 'account',
  'sub': '489257bd-7b04-43ec-a42c-5c09ae54445b',
  'typ': 'Bearer',
  'azp': 'myclient-f312da4e',
  'auth_time': 1571735486,
  'session_state': '28e891db-dedb-4c69-9502-83a985c8a384',
  'acr': '1',
  'allowed-origins': ['https://example.com'],
  'realm_access': {'roles': ['offline_access', 'uma_authorization']},
  'resource_access': {'account': {'roles': ['manage-account',
     'manage-account-links',
     'view-profile']}},
  'scope': 'openid email profile',
  'email_verified': False,
  'preferred_username': 'john-e13b55e3'})

## Query `userinfo`

Check the access token against the `userinfo` endpoint.

In [26]:
r = session.get(
    provider_info["userinfo_endpoint"], 
    headers={"Authorization": "Bearer %s" % access_token}
)
r.raise_for_status()
r.json()

{'sub': '489257bd-7b04-43ec-a42c-5c09ae54445b',
 'email_verified': False,
 'preferred_username': 'john-e13b55e3'}